In [2]:
from bs4 import BeautifulSoup
import re
import requests

In [3]:
response = requests.get('https://comic.naver.com/webtoon/list.nhn?titleId=714834&weekday=fri')
html = response.text
soup = BeautifulSoup(html, 'lxml')

In [4]:
soup

<!DOCTYPE html>
<html lang="ko">
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<title>자취로운 생활 :: 네이버 만화</title>
<meta content="자취로운 생활" property="og:title"/>
<meta content="https://shared-comic.pstatic.net/thumb/webtoon/714834/thumbnail/thumbnail_IMAG19_9542f8f0-8d15-42b8-9cb8-01109bbaa18c.jpg" property="og:image"/>
<meta content="츄카피의 신개념 방구석 서바이벌(?) 자취툰" property="og:description"/>
<meta content="https://comic.naver.com/webtoon/list.nhn?titleId=714834&amp;weekday=fri" property="og:url"/>
<meta content="article" property="og:type"/>
<meta content="네이버 웹툰" property="og:article:author"/>
<meta content="https://comic.naver.com" property="og:article:author:url"/>
<link href="https://ssl.pstatic.net/static/comic/favicon/webtoon_favicon_32x32.ico" rel="shortcut icon" type="image/x-icon"/>
<script type="text/javascript">
		document.domain = "naver.com";
	</script>
<link href="/css/comic/common_20180

In [6]:
title = soup.select_one('td.title')

In [7]:
title

<td class="title">
<a href="/webtoon/detail.nhn?titleId=714834&amp;no=18&amp;weekday=fri" onclick="clickcr(this,'lst.title','714834','18',event)">17. 본家에 가자</a>
</td>

In [42]:
td_list = soup.select('td.title')

In [43]:
td_list

[<td class="title">
 <a href="/webtoon/detail.nhn?titleId=714834&amp;no=18&amp;weekday=fri" onclick="clickcr(this,'lst.title','714834','18',event)">17. 본家에 가자</a>
 </td>, <td class="title">
 <a href="/webtoon/detail.nhn?titleId=714834&amp;no=17&amp;weekday=fri" onclick="clickcr(this,'lst.title','714834','17',event)">16. 츄카피네 놀러 오세요</a>
 </td>, <td class="title">
 <a href="/webtoon/detail.nhn?titleId=714834&amp;no=16&amp;weekday=fri" onclick="clickcr(this,'lst.title','714834','16',event)">15. 탈모 극복(?) 이야기</a>
 </td>, <td class="title">
 <a href="/webtoon/detail.nhn?titleId=714834&amp;no=15&amp;weekday=fri" onclick="clickcr(this,'lst.title','714834','15',event)">14. 자취를 시작한 계기</a>
 </td>, <td class="title">
 <a href="/webtoon/detail.nhn?titleId=714834&amp;no=14&amp;weekday=fri" onclick="clickcr(this,'lst.title','714834','14',event)">13. 혼밥이지만 혼밥이 아니야</a>
 </td>, <td class="title">
 <a href="/webtoon/detail.nhn?titleId=714834&amp;no=13&amp;weekday=fri" onclick="clickcr(this,'lst.title','7

In [47]:
for td in td_list:
    title = td.select_one('a').get_text(strip=True)
    print(title)

17. 본家에 가자
16. 츄카피네 놀러 오세요
15. 탈모 극복(?) 이야기
14. 자취를 시작한 계기
13. 혼밥이지만 혼밥이 아니야
12. 귀찮음이 묻어나는 화
11. 이웃집 사람들
10. 자취는 외로워
9. 혼술이 최고야
8. 야옹야옹야옹시


In [45]:
title

'16. 츄카피네 놀러 오세요'

In [48]:
page = soup.select_one('div.page_wrap')

In [49]:
page

<div class="page_wrap">
<strong class="blind">페이지 이동하기</strong>
<strong class="page"><em class="num_page">1</em><span class="blind"> 현재 페이지</span></strong>
<a class="page" href="/webtoon/list.nhn?titleId=714834&amp;weekday=fri&amp;page=2"><em class="num_page">2</em></a>
<a class="next" href="/webtoon/list.nhn?titleId=714834&amp;weekday=fri&amp;page=2" title="다음 페이지 목록으로 이동"><span class="cnt_page">다음</span><span class="blind"> 페이지 목록으로 이동</span><span class="ico_page"></span></a>
</div>

In [51]:
has_next = page.select_one('span.cnt_page').get_text(strip=True)

In [52]:
has_next

'다음'

In [55]:
result = get_page_episode_dict(page)

ggg 다음


In [56]:
def get_page_episode_dict(page):
    """

    :param page: 크롤링할 페이지
    :return:{
        'has_next': <다음페이지가 있는지>',
        'episode_dict': 'episode_id'를 키, Episode인스턴스를 값으로 쓰는 dict
    }
    """
    response = requests.get(
        self.EPISODE_LIST_BASE_URL,
        params=self.get_episode_list_url_params_dict(page=page)
    )
    soup = BeautifulSoup(response.text, 'lxml')

    # 이 page에 해당하는 Episode들을 담을 dict
    page_episode_dict = OrderedDict()

    # 에피소드 목록이 table.viewList의 각 'tr'요소 하나씩에 해당함
    table = soup.select_one('table.viewList')
    tr_list = table.select('tr')[1:]
    for tr in tr_list:
        # 이 루프 내부의 'tr'하나당 에피소드 하나를 만들어야 함
        try:
            # 데이터 파싱
            td_list = tr.select('td')
            href = td_list[0].select_one('a')['href']
            no = re.search(r'no=(\d+)', href).group(1)
            url_thumbnail = td_list[0].select_one('img')['src']
            title = td_list[1].select_one('a').get_text(strip=True)
            rating = td_list[2].select_one('strong').get_text()
            created_date = td_list[3].get_text(strip=True)
            # 파싱한 데이터로 새 Episode인스턴스 생성
            episode = Episode(
                episode_id=no,
                title=title,
                url_thumbnail=url_thumbnail,
                rating=rating,
                created_date=created_date,
            )
            # 리턴해줄 dict변수에 값 할당
            page_episode_dict[no] = episode
        except:
            # 위 파싱에 실패하는 경우에는 무시 (tr이 의도와 다르게 생겼을때 실패함)
            # 기왕이면 실패 로그를 쌓으면 좋음 (어떤 웹툰의 몇 번째 페이지 몇 번째 row시도중 실패했다를 텍스트 파일에)
            pass

    next_btn = soup.select_one('.paginate a.next')
    return {
        'episode_dict': page_episode_dict,
        'has_next': bool(next_btn),
    }

In [57]:
result = get_page_episode_dict(1)

NameError: name 'self' is not defined